- 2020.07.31 kouui

- 2020.08.07 kouui Grotrian-plot, data-format, wavelength-in-air

---

In [148]:
import numpy as np

%matplotlib tk
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [151]:
import sys
sys.path.append("../../")

%load_ext autoreload
%autoreload 2

from src.Structure import AtomCls
from src.Atomic import Collision

from src.Util import Class

from src.Visual import Grotrian

import src.Constants as Cst
from src.Atomic import LTELib

import Lib2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---

In [169]:
folder = "../../data/atom/Ca_II/"
file_Level     = folder + "Ca_II.Level"
file_Aji = folder + "Einstein_A/Nist.Aji"
file_CEe = folder + "Collisional_Excitation/RH.Electron"
atom = AtomCls.Atom(file_Level, _file_Aji=file_Aji, _file_CEe=file_CEe)

file_CIe = folder + "Collisional_Ionization/RH.Electron"
atom.read_CI(_path_electron=file_CIe)

file_PI  = folder + "Photoionization/RH.Alpha"
atom.read_PI(_path_alpha=file_PI)

file_Rad = folder + "Ca_II.RadiativeLine"
atom.read_RadiativeLine_and_make_Line_Mesh(_path=file_Rad)
atom.make_Cont_Mesh()
atom.read_RadLine_intensity(_folder="../../data/intensity/Ca_II/")

Reading Einstein Aji coefficient from : 
 ../../data/atom/Ca_II/Einstein_A/Nist.Aji
...
Finished.

Reading Electron impact Effective Collisional Strength from : 
 ../../data/atom/Ca_II/Collisional_Excitation/RH.Electron
Finished.

Reading Electron impact Collisional Ionization coefficient from : 
 ../../data/atom/Ca_II/Collisional_Ionization/RH.Electron
Finished.

Reading Photoionization cross section from : 
 ../../data/atom/Ca_II/Photoionization/RH.Alpha
Finished.

Reading Radiative Line information from : 
 ../../data/atom/Ca_II/Ca_II.RadiativeLine
line mesh prepared.
continuum mesh prepared.


In [120]:
Te = 1E4 
Ne = 1E12
#Vt = 5E5
#Vd = 0E5

In [121]:
n_LTE, ni_LTE, nj_LTE = Lib2.ni_nj_LTE(atom,Te,Ne)

In [122]:
res = Lib2.bf_R_rate(atom, Te, ni_LTE, nj_LTE, _Tr=Te)
Rik, Rki_stim, Rki_spon = res

In [123]:
Bij_Jbar, Bji_Jbar = Lib2.B_Jbar(atom, _Tr=Te)

In [124]:
#CEij = Lib2.CEij_rate_coe(atom, Te)
#CIik = Lib2.CIik_rate_coe(atom, Te)
Cij = Lib2.get_Cij(atom, Te)
Cji = Collision.Cij_to_Cji(Cij, ni_LTE, nj_LTE)

In [54]:
n_LTE

array([1.70004652e-02, 2.91957953e-03, 4.33183250e-03, 1.83162623e-04,
       3.51931029e-04, 9.75213029e-01])

## Plots

### Te, Ne v.s. collisional excitation

In [9]:
Te_arr = np.logspace(3.5, 5, 101,endpoint=True)
#Ne_arr = np.logspace(9, 14, 101, endpoint=True)
nTe = Te_arr.shape[0]
#nNe = Ne_arr.shape[0]

CEij_arr = np.zeros((atom.nLine, nTe))

for k in range(nTe):
    CEij_arr[:,k]= Lib2.CEij_rate_coe(atom, Te_arr[k])

In [10]:
names = [f"{atom.Line.w0_AA[i]:.0f}" for i in atom.I_Rad.lineIndex]
index = [i for i in atom.I_Rad.lineIndex]

k = 0

fig, ax = plt.subplots(1,1, figsize=(6,3), dpi=150)
ax.plot(Te_arr, CEij_arr[index[k],:], linewidth=1, label=names[k])

ax.set_xlabel("Te [$K$]")
ax.set_ylabel("CEij [$s^{-1}cm^{3}$]")
ax.legend(loc="best")
plt.show()

### photo-excitation v.s. collisional excitation

In [11]:
# electron temperature
Te_arr = np.logspace(3.5, 4, 101,endpoint=True)
nT = Te_arr.shape[0]
# radiation temperature
Tr_arr = np.logspace(3.5, 4, 101,endpoint=True)


Bij_Jbar_arr = np.zeros((atom.nLine, nT))
CEij_arr = np.zeros((atom.nLine, nT))

for k in range(nT):
    CEij_arr[:,k]= Lib2.CEij_rate_coe(atom, _Te=Te_arr[k])
    
    Bij_Jbar_arr[:,k], _Bji_Jbar = Lib2.B_Jbar(atom, _Tr=Tr_arr[k])

In [12]:
names = [f"{atom.Line.w0_AA[i]:.0f}" for i in atom.I_Rad.lineIndex]
index = [i for i in atom.I_Rad.lineIndex]

k = 0
Ne = 1E12

fig, ax = plt.subplots(1,1, figsize=(6,3), dpi=150)
lns1 = ax.plot(Te_arr, Ne*CEij_arr[index[k],:],"b", linewidth=1, label="Col-Ex")
ax2 = ax.twiny()

lns2 = ax2.plot(Tr_arr, Bij_Jbar_arr[index[k],:],"r", linewidth=1, label="Photo-Ex")

lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc="best")

ax2.set_title(names[k]+"$\AA$", x=0.0, y=1.1)
ax.set_xlabel("Te [$K$]")
ax2.set_xlabel("Tr [$K$]")
ax.set_ylabel("rate [$s^{-1}$]")
ax.set_yscale("log")
plt.show()

### Transition rate plot in Grotrian diagram

#### show Grotrian

In [173]:
gro = Grotrian.Grotrian(_atom=atom, _conf_prefix="1s2.2s2.2p6.3s2.3p6.")
gro.make_fig(_figsize=(10,6),_dpi=150, _f=50)

gro.show_fig()

In [174]:
gro = Grotrian.Grotrian(_atom=atom, _conf_prefix="1s2.2s2.2p6.3s2.3p6.")
gro.make_fig(_figsize=(10,6),_dpi=150, _f=50)

text_selection = ("3968", "3934") # "wavelength", "all"
gro.show_Grotrian(folder + "/Ca_II.Grotrian", _text_selection="wavelength")
gro.show_fig()

#### radiative transition rate

check detail balance

In [334]:
Te = 1E4
Ne = 1E12

#n_LTE0, ni_LTE0, nj_LTE0 = Lib2.ni_nj_LTE_v0(atom,_Te,_Ne)
#_nj_by_ni_Line = nj_LTE0[:atom.nLine] / ni_LTE0[:atom.nLine]
#_nj_by_ni_Cont = nj_LTE0[atom.nLine:] / ni_LTE0[atom.nLine:]

n_LTE , nj_by_ni_Line, nj_by_ni_Cont = Lib2.ni_nj_LTE(atom, Te, Ne)

res = Lib2.bf_R_rate(atom, Te, nj_by_ni_Cont, _Tr=Te)
Rik, Rki_stim, Rki_spon = res
Bij_Jbar, Bji_Jbar = Lib2.B_Jbar(atom, _Tr=Te)
Cij = Lib2.get_Cij(atom, Te)
Cji = Collision.Cij_to_Cji( Cij, np.append(nj_ni_Line,nj_ni_Cont) )

print("Line Transitions", end="\n")
for _lineIdx in range(atom.nLine):
    _nj_by_ni = nj_by_ni_Line[_lineIdx]
    print(f"wavelength : {atom.Line.w0_AA[_lineIdx]:.0f}[A]", end="    ")
    print("net rate = ", end="")
    print((atom.Line.AJI[_lineIdx] + Bji_Jbar[_lineIdx]) * _nj_by_ni \
          - Bij_Jbar[_lineIdx])

print()
print("Continuum Transitions", end="\n")
for _contIdx in range(atom.nCont):
    _nj_by_ni = nj_by_ni_Cont[_contIdx]
    print(f"wavelength : {atom.Cont.w0_AA[_contIdx]:.0f}[A]", end="    ")
    print("net rate = ", end="")
    print((Rki_spon[_contIdx] + Rki_stim[_contIdx]) * _nj_by_ni \
          - Rik[_contIdx])


Line Transitions
wavelength : 7326[A]    net rate = 0.0
wavelength : 7293[A]    net rate = 0.0
wavelength : 3970[A]    net rate = -1.3969838619232178e-09
wavelength : 3935[A]    net rate = 0.0
wavelength : 1647849[A]    net rate = 0.0
wavelength : 8665[A]    net rate = 2.3283064365386963e-10
wavelength : 8500[A]    net rate = -5.820766091346741e-11
wavelength : 8710[A]    net rate = 0.0
wavelength : 8544[A]    net rate = 0.0
wavelength : 448649[A]    net rate = 0.0

Continuum Transitions
wavelength : 1044[A]    net rate = 3.552713678800501e-15
wavelength : 1218[A]    net rate = 6.821210263296962e-13
wavelength : 1219[A]    net rate = 6.821210263296962e-13
wavelength : 1417[A]    net rate = 4.547473508864641e-13
wavelength : 1422[A]    net rate = 2.7284841053187847e-12


In [336]:
gro.make_fig(_figsize=(10,6),_dpi=150, _f=50)

cmap = plt.get_cmap('cool')
norm = LogNorm(1E-3, 1E5, clip=True)


Te = 8E3
Ne = 1E12
Tr = 6E3

#n_LTE, ni_LTE, nj_LTE = Lib2.ni_nj_LTE_v0(atom,Te,Ne)
#nj_by_ni_Line = nj_LTE[:atom.nLine] / ni_LTE[:atom.nLine]
#nj_by_ni_Cont = nj_LTE[atom.nLine:] / ni_LTE[atom.nLine:]

n_LTE , nj_by_ni_Line, nj_by_ni_Cont = Lib2.ni_nj_LTE(atom, Te, Ne)

res = Lib2.bf_R_rate(atom, Te, nj_by_ni_Cont, _Tr=Tr)
Rik, Rki_stim, Rki_spon = res
Bij_Jbar, Bji_Jbar = Lib2.B_Jbar(atom, _Tr=Tr)

rate = np.append( Bji_Jbar*nj_by_ni_Line + 
                  atom.Line.AJI*nj_by_ni_Line - 
                  Bij_Jbar,
                  Rki_stim*nj_by_ni_Cont +
                  Rki_spon*nj_by_ni_Cont - 
                  Rik )

idxI = np.append( atom.Line.idxI, atom.Cont.idxI )
idxJ = np.append( atom.Line.idxJ, atom.Cont.idxJ )

gro.show_transition_rate(_idxI=idxI, 
                         _idxJ=idxJ, 
                         _rate=rate, 
                         _direction="j->i",
                         _cmap=cmap, _norm=norm)

#### collisional transition rate

In [20]:
gro.make_fig(_figsize=(10,6),_dpi=150, _f=50)

cmap = plt.get_cmap('cool')
norm = LogNorm(1E3, 1E8, clip=True)

Ne = 1E12
rate = (Cji - Cij)*Ne
idxI = np.append( atom.Line.idxI, atom.Cont.idxI )
idxJ = np.append( atom.Line.idxJ, atom.Cont.idxJ )

gro.show_transition_rate(_idxI=idxI, 
                         _idxJ=idxJ, 
                         _rate=rate, 
                         _direction="j->i",
                         _cmap=cmap, _norm=norm)

#### transition plot respective to a single level

In [21]:
gro.make_fig(_figsize=(10,6),_dpi=150, _f=50)

cmap = plt.get_cmap('cool')
norm = LogNorm(1E3, 1E8, clip=True)

Ne = 1E12
rate = (Cji - Cij)*Ne
idxI = np.append( atom.Line.idxI, atom.Cont.idxI )
idxJ = np.append( atom.Line.idxJ, atom.Cont.idxJ )

gro.show_transition_rate(_idxI=idxI, 
                         _idxJ=idxJ, 
                         _rate=rate, 
                         _direction="j->i",
                         _cmap=cmap, _norm=norm,
                         _level_ctj_without_prefix=("4p","2P","3/2"))

---

# backup

In [333]:
_l = 8
wave = atom.Line.w0[_l]
_Te = 1E4
_chi = Cst.h_*Cst.c_/wave
_gi = atom.Level.g[ atom.Line.idxI[_l] ]
_gj = atom.Level.g[ atom.Line.idxJ[_l] ]
_Aji = 1E7

_Bji, _Bij = LTELib.EinsteinA_to_EinsteinBs_cm(_Aji,wave,_gi, _gj)
_nj_by_ni = float(_gj) / float(_gi) * np.exp(-_chi/Cst.k_/_Te)

_Jbar = LTELib.Planck_cm(wave,_Te)

print( (_Aji + _Bji*_Jbar) * _nj_by_ni - _Bij*_Jbar )

-2.3283064365386963e-10


In [276]:
_lineIdx = 6
_Te = 1E4              # [K]
_chi = Cst.h_*atom.Line.f0[_lineIdx]  # [erg]
_gi = atom.Level.g[ atom.Line.idxI[_lineIdx] ]
_gj = atom.Level.g[ atom.Line.idxJ[_lineIdx] ]
_Aji = atom.Line.AJI[_lineIdx]            # [cm^{-1}]

n_LTE, ni_LTE, nj_LTE = Lib2.ni_nj_LTE(atom,Te,Ne)

_w0 = Cst.h_ * Cst.c_ / _chi  # [cm]
print(f"wavelength : {_w0*1E8:.0f}[A]")
_Bji, _Bij = LTELib.EinsteinA_to_EinsteinBs_cm(_Aji,_w0,_gi, _gj)
#_nj_by_ni = _gj / _gi * np.exp(-_chi/Cst.k_/_Te)
#_nj_by_ni = LTELib.Boltzmann_distribution(_gi, _gj, _chi, _Te)

#n_LTE, ni_LTE, nj_LTE = Lib2.ni_nj_LTE_v0(atom,_Te,_Ne)
#_ni = ni_LTE[_lineIdx]
#_nj = nj_LTE[_lineIdx]
#_nj_by_ni = _nj/_ni
#_nj_by_ni = LTELib.Boltzmann_distribution(_gi, _gj, _chi, _Te)
_nj_by_ni = nj_by_ni_Line[_lineIdx]

_Jbar = LTELib.Planck_cm(_w0,_Te)
print((_Aji + _Bji*_Jbar) * _nj_by_ni - _Bij*_Jbar)

wavelength : 8500[A]
-5.820766091346741e-11
